# Introduction
Ce notebook a pour objectif de réaliser diverses opérations sur les usagers de la médiathèque :

In [1]:
import pandas as pd
from datetime import datetime

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

In [3]:
query = """SELECT * 
FROM koha_prod.borrowers b"""

In [4]:
items = pd.read_sql(query, db_conn)

In [5]:
items.columns #avant la traduction

Index(['borrowernumber', 'cardnumber', 'surname', 'firstname', 'title',
       'othernames', 'initials', 'streetnumber', 'streettype', 'address',
       'address2', 'city', 'state', 'zipcode', 'country', 'email', 'phone',
       'mobile', 'fax', 'emailpro', 'phonepro', 'B_streetnumber',
       'B_streettype', 'B_address', 'B_address2', 'B_city', 'B_state',
       'B_zipcode', 'B_country', 'B_email', 'B_phone', 'dateofbirth',
       'branchcode', 'categorycode', 'dateenrolled', 'dateexpiry',
       'date_renewed', 'gonenoaddress', 'lost', 'debarred', 'debarredcomment',
       'contactname', 'contactfirstname', 'contacttitle', 'borrowernotes',
       'relationship', 'sex', 'password', 'flags', 'userid', 'opacnote',
       'contactnote', 'sort1', 'sort2', 'altcontactfirstname',
       'altcontactsurname', 'altcontactaddress1', 'altcontactaddress2',
       'altcontactaddress3', 'altcontactstate', 'altcontactzipcode',
       'altcontactcountry', 'altcontactphone', 'smsalertnumber',
       '

In [6]:
items = items.rename(columns={"cardnumber":"numéro de carte",
                              "surname":"nom de famille",
                              "firstname":"prénom",
                              "title":"civilité",
                              "streetnumber":"numéro de rue",
                              "streettype":"type de rue",
                              "address":"adresse",
                              "city":"ville",
                              "state":"province",
                              "country":"pays",
                              "phone":"numéro de téléphone fixe",
                              "mobile":"numéro de téléphone portable",
                              "dateofbirth":"date de naissance",
                              "branchcode":"site de rattachement",
                              "categorycode":"type de carte",
                              "dateenrolled":"date d'inscription",
                              "dateexpiry":"date d'expiration",
                              "date_renewed":"date de renouvellement",
                              "gonenoaddress":"parti sans laisser d'adresse",
                              "lost":"carte perdue",
                              "debarred":"date de suspension",
                              "flags":"permissions",
                              "debarredcomment":"commentaire de suspension",
                              "contactname":"nom du garant",
                              "updated_on":"date dernier changement",
                              "lastseen":"date dernière transaction"})
                              

In [7]:
items.columns #après la traduction

Index(['borrowernumber', 'numéro de carte', 'nom de famille', 'prénom',
       'civilité', 'othernames', 'initials', 'numéro de rue', 'type de rue',
       'adresse', 'address2', 'ville', 'province', 'zipcode', 'pays', 'email',
       'numéro de téléphone fixe', 'numéro de téléphone portable', 'fax',
       'emailpro', 'phonepro', 'B_streetnumber', 'B_streettype', 'B_address',
       'B_address2', 'B_city', 'B_state', 'B_zipcode', 'B_country', 'B_email',
       'B_phone', 'date de naissance', 'site de rattachement', 'type de carte',
       'date d'inscription', 'date d'expiration', 'date de renouvellement',
       'parti sans laisser d'adresse', 'carte perdue', 'date de suspension',
       'commentaire de suspension', 'nom du garant', 'contactfirstname',
       'contacttitle', 'borrowernotes', 'relationship', 'sex', 'password',
       'permissions', 'userid', 'opacnote', 'contactnote', 'sort1', 'sort2',
       'altcontactfirstname', 'altcontactsurname', 'altcontactaddress1',
       'al

# DÉFINITION DE QUELQUES VARIABLES IMPORTANTES
## Transformation des colonnes au format `Date`

In [8]:
items["date de naissance"] = pd.to_datetime(items["date de naissance"])

In [9]:
items["date d'inscription"] = pd.to_datetime(items["date d'inscription"])

In [10]:
items["date d'expiration"] = pd.to_datetime(items["date d'expiration"])

In [11]:
items["date de renouvellement"] = pd.to_datetime(items["date de renouvellement"])

## Colonnes à exporter

In [12]:
colonnes_a_exporter = ["numéro de carte","nom de famille","prénom","civilité","numéro de rue","type de rue","adresse","ville","province","pays","numéro de téléphone fixe","numéro de téléphone portable","email","date de naissance","site de rattachement","type de carte","date d'inscription","date d'expiration","date de renouvellement","parti sans laisser d'adresse","carte perdue","date de suspension","permissions","commentaire de suspension","nom du garant","date dernier changement","date dernière transaction"]

# ANALYSE DES ANOMALIES SUR LES CARTES D'ADHÉRENTS
## Analyse sur les données manquantes


Lors de l'inscription d'un adhérent, de nombreux champs sont à renseigner. Certains champs sont *obligatoires* et d'autres non. Ce sont sur les champs non-obligatoires que nous allons nous intéresser ici. En voici la liste : 
* [ ] prénom
* [ ] nom de famille du garant (REQUÊTE SPÉCIALE !)
* [ ] ni fixe, ni mobile, ni adresse mail
* [ ] ...
* [ ] ...
* [ ] ...
* [ ] ...
* [ ] ...

### Numéro de téléphone

In [21]:
telephone_ko = items[~items["numéro de téléphone fixe"].isna()]
telephone_ko = telephone_ko[~telephone_ko["numéro de téléphone portable"].isna()] #on est obligé de passer par là, la fonction str.contains n'accepte pas les NaN

In [23]:
telephone_ko = telephone_ko[(~telephone_ko["numéro de téléphone fixe"].str.contains("\w",regex=True))]
telephone_ko = telephone_ko[(~telephone_ko["numéro de téléphone portable"].str.contains("\w",regex=True))]

In [24]:
len(telephone_ko)

5338

## Email

In [118]:
email_ko = items[~items["email"].isna()]
email_ko = email_ko[~email_ko["email"].str.contains("\w",regex=True)]